<a href="https://colab.research.google.com/github/GenaroHacker/crystal_clear/blob/main/filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!git clone https://github.com/GenaroHacker/crystal_clear.git
%run crystal_clear/sql.ipynb

fatal: destination path 'crystal_clear' already exists and is not an empty directory.


In [15]:
#@title Finctions

def delete_paradigm(paradigm_id):
  list_of_tuples = ReadRecords("BaseParadigms","TABLE_PARADIGMS")
  tablename_todelete = ""
  for paradigm in list_of_tuples:
    if paradigm[0] == paradigm_id:
      tablename_todelete = paradigm[1]
  RunCommand("BaseParadigms","DROP TABLE " + tablename_todelete)

def sort_paradigm(paradigm_id):
  list_of_tuples = ReadRecords("BaseParadigms","TABLE_PARADIGMS")
  tablename_tosort = ""
  for paradigm in list_of_tuples:
    if paradigm[0] == paradigm_id:
      tablename_tosort = paradigm[1]
  #Read all records
  list_of_tuples = ReadRecords("BaseParadigms",tablename_tosort)
  prdgms_to_sort = []
  for record in list_of_tuples:
    prdgms_to_sort.append(record[1])
  def manual_sort(items_to_sort):
    import functools
    def ask_user_cmp(item1, item2):
        while True:
            print(f"[{item1}](1) or [{item2}](2) ?")
            cmp = input("--> ")
            if cmp == "1":
                return 1
            if cmp == "2":
                return -1
            print("1 or 2, please!")
    ask_user_key = functools.cmp_to_key(ask_user_cmp)
    items_to_sort.sort(key=ask_user_key, reverse=True)
    return items_to_sort
  prdgms_sorted = manual_sort(prdgms_to_sort)
  RunCommand("BaseParadigms","DELETE FROM " + tablename_tosort)
  RunCommand("BaseParadigms","DELETE FROM sqlite_sequence WHERE name='" + tablename_tosort + "'")
  for phrase in prdgms_sorted:
    InsertRecord("BaseParadigms","INSERT INTO " + tablename_tosort + " VALUES (NULL,'" + phrase + "')")

def update_phrase(paradigm_id, phrase_id, new_phrase):
  list_of_tuples = ReadRecords("BaseParadigms","TABLE_PARADIGMS")
  tablename = ""
  for paradigm in list_of_tuples:
    if paradigm[0] == paradigm_id:
      tablename = paradigm[1]
  UpdateRecord("BaseParadigms","UPDATE " + tablename + " SET PHRASE='" + new_phrase + "' WHERE ID=" + str(phrase_id))

def delete_phrase(paradigm_id,phrase_id):
  list_of_tuples = ReadRecords("BaseParadigms","TABLE_PARADIGMS")
  tablename = ""
  for paradigm in list_of_tuples:
    if paradigm[0] == paradigm_id:
      tablename = paradigm[1]
  last_record_id = ReadLastRecord("BaseParadigms",tablename)[0]
  RemoveRecord("BaseParadigms","DELETE FROM " + tablename + " WHERE ID=" + str(phrase_id))
  for id in range(phrase_id + 1,last_record_id + 1):
    UpdateRecord("BaseParadigms","UPDATE " + tablename + " SET ID='" + str(id-1) + "' WHERE ID=" + str(id))


In [16]:
#delete_paradigm(2)
#sort_paradigm(3)
#update_phrase(3,1,"lol")
#delete_phrase(1,1)

In [22]:
#@title Execution


print("Select the paradigm you want to filter or type 0 to select a random paradigm")
list_of_paradigms = ReadRecords("BaseParadigms","TABLE_PARADIGMS")
print_table(list_of_paradigms)

ids = []
for i in list_of_paradigms:
  ids.append(i[0])


user_choice = ""
while type(user_choice) != int:
  try:
    user_choice = int(input())
    if user_choice not in ids and user_choice != 0:
      user_choice = ""
      int("I just hope that no ValueError occurs")
  except ValueError:
    print("Only IDs available in the table or 0 are allowed")
if user_choice == 0:
  from random import choice
  user_choice = choice(ids)
  
selected_paradigm_id = user_choice
tablename = list_of_paradigms[selected_paradigm_id-1][1]
print()
print(tablename)
list_of_phrases = ReadRecords("BaseParadigms",tablename)
print_table(list_of_phrases)

print()
print("What do you want to do with [" + tablename + "]?")

#delete_paradigm(2)
#sort_paradigm(3)
#update_phrase(3,1,"lol")
#delete_phrase(1,1)

print(" 1 --> sort paradigm")
print(" 2 --> update phrase")
print(" 3 --> delete phrase")
print(" 4 --> delete paradigm")
selected_action = input()
while selected_action not in ("1", "2", "3", "4"):
  print("Option not available.")
  selected_action = input()

if selected_action == "1":
  #sort_paradigm(3)
  pass

elif selected_action == "2":
  #update_phrase(3,1,"lol")
  pass

elif selected_action == "3":
  #delete_phrase(1,1)
  pass

elif selected_action == "4":
  #delete_paradigm(2)
  pass

Select the paradigm you want to filter or type 0 to select a random paradigm
1 | Directivo | 3 | 0 | 
2 | Coach     | 3 | 0 | 
3 | Gracioso  | 3 | 0 | 
d
Only IDs available in the table or 0 are allowed
2

Coach
1 | que tenes en mente?                  | 
2 | cual es el mayor reto que enfrentas? | 
3 | cual es el costo de decir que si?    | 

What do you want to do with [Coach]?
 1 --> sort paradigm
 2 --> update phrase
 3 --> delete phrase
 4 --> delete paradigm
v
Option not available.
d
Option not available.
g
Option not available.
th
Option not available.
1
